In [18]:
import numpy as np
import cv2
import os
from skimage.filters.rank import entropy
import pandas as pd

#  Phase 1: Feature Extraction

## Load Images from Dataset
This function loads images, resizes them, and stores them in arrays.

In [20]:
def load_images(folder, img_size=(256, 256)):
    images = []
    labels = []
    for class_name in os.listdir(folder):
        class_path = os.path.join(folder, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, img_size)
                images.append(img)
                labels.append(class_name)
    return np.array(images), np.array(labels)

## Feature Extraction
The following function extracts various features including color, texture, edge density, and statistical properties.


In [21]:
def extract_features(images):
    """
    Extracts various features from images including color, texture, edge density, 
    and statistical properties.
    """
    features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Color Features: Mean values of R, G, B channels
        mean_colors = img.mean(axis=(0, 1))

        # Texture Feature: Variance of Laplacian (sharpness measure)
        lbp = cv2.Laplacian(gray, cv2.CV_64F).var()

        # Edge Feature: Edge density using Canny edge detection
        edges = cv2.Canny(gray, 100, 200)
        edge_density = np.mean(edges)

        # Statistical Feature: Combined Mean & Variance of pixel intensity
        stat_features = np.mean(gray) + np.var(gray)

        # Color Variance: Measures the diversity of colors in the image
        color_variance = np.var(img, axis=(0, 1)).mean()

        # Texture Roughness: Measures the roughness of the texture
        roughness = np.mean(np.abs(np.diff(gray.astype(np.float32))))

        # Image Entropy: Measures randomness/complexity in the image
        entropy_value = entropy(gray, np.ones((3,3))).mean()

        # Append features to the list
        features.append([*mean_colors, lbp, edge_density, stat_features, color_variance, roughness, entropy_value])

    return np.array(features)

## Process Images and Save Features
Now, we load the images, extract features, and save them into a CSV file for further analysis.


In [22]:
# Load images from dataset
image_folder = "./dataset/train"
images, labels = load_images(image_folder)

# Extract features
features = extract_features(images)

# Save extracted features into a CSV file for further analysis
df = pd.DataFrame(features, columns=["R", "G", "B", "LBP", "Edges", "Stat_Feature", "Color_Variance", "Roughness", "Entropy"])
df.to_csv("features.csv", index=False)

print("✅ Feature extraction completed and saved!")

✅ Feature extraction completed and saved!


## Summary of Extracted Features  

Below is a summary of all extracted features along with their descriptions:

|  **Feature Name** |  **Description** |
|--------------|-------------------------------------------------------------|
| **R, G, B** | Average pixel intensity in Red, Green, and Blue channels (color features) |
| **LBP (Laplacian Variance)** | Measures texture sharpness and contrast (higher = sharper texture) |
| **Edge Density** | Number of edges detected using the Canny edge detector (higher = more edges) |
| **Stat_Feature** | Combined Mean & Variance of pixel intensities (brightness & contrast) |
| **Color Variance** | Measures the diversity of colors in the image (higher = more colorful) |
| **Texture Roughness** | Measures surface roughness using pixel intensity differences (higher = rougher) |
| **Entropy** | Measures randomness & complexity in the image (higher = more complex texture) |

---
### ✅ **Key Insights**
✔ **Higher Edge Density** → More detailed images with strong contours.  
✔ **Higher Entropy** → More complex images with diverse textures.  
✔ **Higher Color Variance** → More colorful and vibrant images.  
✔ **Higher Roughness** → More irregular and noisy textures.  

These features will be used in the next step for **Feature Selection (Phase 2).**  
**Let’s move to the next step!**
